In [87]:
import pandas as pd
import numpy as np
import re,yaml
import itertools as it
import networkx as nx

In [88]:
repL4L="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/"
fL4L="L4L-paradigmes-tail.csv"
paradigmes=pd.read_csv(repL4L+fL4L,sep=";",encoding="utf8")

In [89]:
cols=paradigmes.columns.tolist()
cases=cols[2:]
print ", ".join(cases)

pP, pI1P, ps1S, ps1P, ps3P, ps3S, pc2S, pc2P, ii3P, inf, ii3S, ai2P, ai2S, is2P, pi1P, pi1S, pi3P, ii1S, ii1P, is2S, pi3S, fi2S, fi2P, pI2S, pI2P, ps2S, ps2P, ppFS, ppFP, pc1S, pc1P, ai1P, pc3P, pc3S, ai1S, is3S, ii2S, ai3S, ai3P, ii2P, pi2P, ppMP, pi2S, ppMS, is3P, is1P, is1S, fi1S, fi1P, fi3P, fi3S


In [90]:
for c in cases:
    display(paradigmes[[c]].dropna().head(1))


,pP
24,bursikOtâ


,pI1P
2,rEêstalô


,ps1S
9,amêsis


,ps1P
65,plOzjô


,ps3P
1,Evid


...

,is1S
5,d6vês


,fi1S
2,rEêstal6rE


,fi1P
1,Evid6rô


,fi3P
10,dEvErs6rô


,fi3S
9,amêsira


In [137]:
g=nx.Graph()
for (c1,c2) in it.combinations(cases, 2):
    c1Val=paradigmes[c1].notnull()
    c2Val=paradigmes[c2].notnull()
    incompatibles=paradigmes[c1Val & c2Val & (paradigmes[c1]!=paradigmes[c2])][[c1,c2]]
    compatibles=paradigmes[c1Val & c2Val & (paradigmes[c1]==paradigmes[c2])][[c1,c2]]
    if len(incompatibles)==0 and len(compatibles)>0:
        g.add_edge(c1,c2,weight=len(compatibles))
cliques=list(nx.find_cliques(g))

In [200]:
cliques=sorted(cliques, key=len, reverse=True)
wCliques=(cliques[:])

In [201]:
syncretismes=[]
syncretiques=set()

def cleanCliques(lCliques):
    sCases=set(sum(lCliques,[]))
    syncretiques.update(sCases)
    for c in lCliques:
        wCliques.remove(c)
    for c in sCases:
        for w in wCliques:
            if c in w:
                w.remove(c)
    return
                

def addCliques(length):
    conflits=[]
    lCliques=[x for x in cliques if len(x)==length]
    if len(lCliques)>1:
        for (c1,c2) in it.combinations(lCliques, 2):
            inter=set(c1).intersection(set(c2))
            if inter:
                conflits.append([c1,c2])
    if conflits:
        ajouts=[]
        for c in lCliques:
            noConflit=True
            for conflit in conflits:
                if c in conflit:
                    noConflit=False
            if noConflit:
                ajouts.append(c)
        syncretismes.extend(ajouts)
        cleanCliques(ajouts)
        for c1,c2 in conflits:
            sC1=set(c1)
            sC2=set(c2)
            pivot=list(sC1.intersection(sC2))[0]
            dC1=sC1.difference(sC2)
            wC1=sum(g[pivot][c]["weight"] for c in dC1)
            dC2=sC2.difference(sC1)
            wC2=sum(g[pivot][c]["weight"] for c in dC2)
            print c1,wC1,c2,wC2
    else:
        syncretismes.extend(lCliques)
        cleanCliques(lCliques)
    return

In [202]:
addCliques(5)
syncretismes,syncretiques,wCliques

([[u'ps3P', u'ps2S', u'ps3S', u'pi3P', u'ps1S'],
  [u'pc2S', u'pc3S', u'pc3P', u'pc1S', u'fi1S']],
 {u'fi1S',
  u'pc1S',
  u'pc2S',
  u'pc3P',
  u'pc3S',
  u'pi3P',
  u'ps1S',
  u'ps2S',
  u'ps3P',
  u'ps3S'},
 [[u'ii1S', u'ii3S', u'ii2S', u'ii3P'],
  [u'pi3S', u'pi1S', u'pI2S', u'pi2S'],
  [u'is3P', u'is2S', u'is1S'],
  [u'is3S', u'ai3S', u'ai2S'],
  [u'ppMS', u'ppMP'],
  [u'ppMS', u'ps2P'],
  [u'ppFS', u'ppFP'],
  [u'fi2S', u'fi3S'],
  [u'fi3P', u'fi1P'],
  [u'ppMS'],
  []])

In [203]:
addCliques(4)
syncretismes,syncretiques,wCliques

([[u'ps3P', u'ps2S', u'ps3S', u'pi3P', u'ps1S'],
  [u'pc2S', u'pc3S', u'pc3P', u'pc1S', u'fi1S'],
  [u'ii1S', u'ii3S', u'ii2S', u'ii3P'],
  [u'pi3S', u'pi1S', u'pI2S', u'pi2S']],
 {u'fi1S',
  u'ii1S',
  u'ii2S',
  u'ii3P',
  u'ii3S',
  u'pI2S',
  u'pc1S',
  u'pc2S',
  u'pc3P',
  u'pc3S',
  u'pi1S',
  u'pi2S',
  u'pi3P',
  u'pi3S',
  u'ps1S',
  u'ps2S',
  u'ps3P',
  u'ps3S'},
 [[u'is3P', u'is2S', u'is1S'],
  [u'is3S', u'ai3S', u'ai2S'],
  [u'ppMS', u'ppMP'],
  [u'ppMS', u'ps2P'],
  [u'ppFS', u'ppFP'],
  [u'fi2S', u'fi3S'],
  [u'fi3P', u'fi1P'],
  [u'ppMS'],
  []])

In [204]:
addCliques(3)
syncretismes,syncretiques,wCliques

([[u'ps3P', u'ps2S', u'ps3S', u'pi3P', u'ps1S'],
  [u'pc2S', u'pc3S', u'pc3P', u'pc1S', u'fi1S'],
  [u'ii1S', u'ii3S', u'ii2S', u'ii3P'],
  [u'pi3S', u'pi1S', u'pI2S', u'pi2S'],
  [u'is3P', u'is2S', u'is1S'],
  [u'is3S', u'ai3S', u'ai2S']],
 {u'ai2S',
  u'ai3S',
  u'fi1S',
  u'ii1S',
  u'ii2S',
  u'ii3P',
  u'ii3S',
  u'is1S',
  u'is2S',
  u'is3P',
  u'is3S',
  u'pI2S',
  u'pc1S',
  u'pc2S',
  u'pc3P',
  u'pc3S',
  u'pi1S',
  u'pi2S',
  u'pi3P',
  u'pi3S',
  u'ps1S',
  u'ps2S',
  u'ps3P',
  u'ps3S'},
 [[u'ppMS', u'ppMP'],
  [u'ppMS', u'ps2P'],
  [u'ppFS', u'ppFP'],
  [u'fi2S', u'fi3S'],
  [u'fi3P', u'fi1P'],
  [u'ppMS'],
  []])

In [205]:
addCliques(2)
syncretismes,syncretiques,wCliques

[u'ppMS', u'ppMP'] 302 [u'ppMS', u'ps2P'] 1


([[u'ps3P', u'ps2S', u'ps3S', u'pi3P', u'ps1S'],
  [u'pc2S', u'pc3S', u'pc3P', u'pc1S', u'fi1S'],
  [u'ii1S', u'ii3S', u'ii2S', u'ii3P'],
  [u'pi3S', u'pi1S', u'pI2S', u'pi2S'],
  [u'is3P', u'is2S', u'is1S'],
  [u'is3S', u'ai3S', u'ai2S'],
  [u'ppFS', u'ppFP'],
  [u'fi2S', u'fi3S'],
  [u'fi3P', u'fi1P']],
 {u'ai2S',
  u'ai3S',
  u'fi1P',
  u'fi1S',
  u'fi2S',
  u'fi3P',
  u'fi3S',
  u'ii1S',
  u'ii2S',
  u'ii3P',
  u'ii3S',
  u'is1S',
  u'is2S',
  u'is3P',
  u'is3S',
  u'pI2S',
  u'pc1S',
  u'pc2S',
  u'pc3P',
  u'pc3S',
  u'pi1S',
  u'pi2S',
  u'pi3P',
  u'pi3S',
  u'ppFP',
  u'ppFS',
  u'ps1S',
  u'ps2S',
  u'ps3P',
  u'ps3S'},
 [[u'ppMS', u'ppMP'], [u'ppMS', u'ps2P'], [u'ppMS'], []])

In [206]:
syncretismes.append([u'ppMS', u'ppMP'])
syncretiques.union(set([u'ppMS', u'ppMP']))
# syncretiques

{u'ai2S',
 u'ai3S',
 u'fi1P',
 u'fi1S',
 u'fi2S',
 u'fi3P',
 u'fi3S',
 u'ii1S',
 u'ii2S',
 u'ii3P',
 u'ii3S',
 u'is1S',
 u'is2S',
 u'is3P',
 u'is3S',
 u'pI2S',
 u'pc1S',
 u'pc2S',
 u'pc3P',
 u'pc3S',
 u'pi1S',
 u'pi2S',
 u'pi3P',
 u'pi3S',
 u'ppFP',
 u'ppFS',
 u'ppMP',
 u'ppMS',
 u'ps1S',
 u'ps2S',
 u'ps3P',
 u'ps3S'}

In [207]:
dExpansion={s[0]:s for s in syncretismes}
dExpansion

{u'fi2S': [u'fi2S', u'fi3S'],
 u'fi3P': [u'fi3P', u'fi1P'],
 u'ii1S': [u'ii1S', u'ii3S', u'ii2S', u'ii3P'],
 u'is3P': [u'is3P', u'is2S', u'is1S'],
 u'is3S': [u'is3S', u'ai3S', u'ai2S'],
 u'pc2S': [u'pc2S', u'pc3S', u'pc3P', u'pc1S', u'fi1S'],
 u'pi3S': [u'pi3S', u'pi1S', u'pI2S', u'pi2S'],
 u'ppFS': [u'ppFS', u'ppFP'],
 u'ppMS': [u'ppMS', u'ppMP'],
 u'ps3P': [u'ps3P', u'ps2S', u'ps3S', u'pi3P', u'ps1S']}

In [208]:
if "fi2S" in dExpansion:
    if "fi3S" in dExpansion["fi2S"]:
        dExpansion[u"fi3S"]=dExpansion.pop("fi2S")
if "ii1S" in dExpansion:
    if "ii3S" in dExpansion["ii1S"]:
        dExpansion[u"ii3S"]=dExpansion.pop("ii1S")
if "pc2S" in dExpansion:
    if "pc3S" in dExpansion["pc2S"]:
        dExpansion[u"pc3S"]=dExpansion.pop("pc2S")
if "pi2S" in dExpansion:
    if "pi3S" in dExpansion["pi2S"]:
        dExpansion[u"pi3S"]=dExpansion.pop("pi2S")
if "ppMP" in dExpansion:
    if "ppMS" in dExpansion["ppMP"]:
        dExpansion[u"ppMS"]=dExpansion.pop("ppMP")

In [209]:
dExpansion

{u'fi3P': [u'fi3P', u'fi1P'],
 u'fi3S': [u'fi2S', u'fi3S'],
 u'ii3S': [u'ii1S', u'ii3S', u'ii2S', u'ii3P'],
 u'is3P': [u'is3P', u'is2S', u'is1S'],
 u'is3S': [u'is3S', u'ai3S', u'ai2S'],
 u'pc3S': [u'pc2S', u'pc3S', u'pc3P', u'pc1S', u'fi1S'],
 u'pi3S': [u'pi3S', u'pi1S', u'pI2S', u'pi2S'],
 u'ppFS': [u'ppFS', u'ppFP'],
 u'ppMS': [u'ppMS', u'ppMP'],
 u'ps3P': [u'ps3P', u'ps2S', u'ps3S', u'pi3P', u'ps1S']}

In [210]:
omps=paradigmes[[c for c in cols if c not in syncretiques]].copy()

In [211]:
def fusionFormes(row):
    result=np.nan
    for c in row:
        if c==c:
            result=c
            break
    return result

In [212]:
# paradigmes[["ppMS","ppMP"]].apply(fusionFormes,axis=1)

In [213]:
for s in dExpansion:
    print s,dExpansion[s]
    omps[s]=paradigmes[dExpansion[s]].apply(fusionFormes,axis=1)

fi3S [u'fi2S', u'fi3S']
pc3S [u'pc2S', u'pc3S', u'pc3P', u'pc1S', u'fi1S']
ppMS [u'ppMS', u'ppMP']
ppFS [u'ppFS', u'ppFP']
fi3P [u'fi3P', u'fi1P']
is3P [u'is3P', u'is2S', u'is1S']
is3S [u'is3S', u'ai3S', u'ai2S']
ii3S [u'ii1S', u'ii3S', u'ii2S', u'ii3P']
ps3P [u'ps3P', u'ps2S', u'ps3S', u'pi3P', u'ps1S']
pi3S [u'pi3S', u'pi1S', u'pI2S', u'pi2S']


In [214]:
with open(repL4L+fL4L.replace(".csv","-omp.yaml"),"w") as outFile:
    yaml.safe_dump(dExpansion,outFile)

In [215]:
omps.dropna(thresh=3).to_csv(repL4L+fL4L.replace(".csv","-omp.csv"),encoding="utf8",sep=";",index=None)